# `ULO`  

#### GENERAL FRAMEWORK 

* discrete pointwise minimum of <i>pieces</i> with common <i>functional constraints</i>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "retina"
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
import cvxpy as cp
clear_output()
rs_seed = 120824#130824
np.random.seed(rs_seed)
import time
import itertools
import pandas as pd

<b>Problem</b>

$\omega \in (0,1]$

$$ \min_{x \in \mathcal{X}}\,\frac{\tau}{2}||x||^2 + \omega \cdot \max_{\tilde{i} \in [n]} h^{(\tilde{i})}(x) + (1-\omega) \cdot \min_{i \in [n]} h^{(i)}(x)$$
$$\max_{j \in [m]}\,c^{(j)}(x) \leq 0$$

In [2]:
## parameters 
TOL_REL_DEF = 0
TOL_ABS_DEF = 0
TOL_SIGNIF_CSTR_DEF = 1e-3
MAX_ITER_DEF = 1e3
MAX_TIME_ABS_DEF = 240
verb_def = False

name = 'bign_experiment'

In [3]:
'''
@comment
'''
def RAM_OPO(list_q_h,list_q_c,list_kappa_h,list_kappa_c,omega=0,M=1e4,tau=0,init=0,rho=1e-3,domain_size=10,extra_verb=verb_def,\
        MAX_TIME=MAX_TIME_ABS_DEF,\
        MAX_ITER=MAX_ITER_DEF,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF):
    
    m = list_q_c.shape[0]
    n,d = list_q_h.shape
    
    ###### EVAL TOOLS
    F = lambda u,eta: tau/2*sum(u**2) + M*eta + omega*np.max(list_q_h@u+list_kappa_h) + (1-omega)*np.min(list_q_h@u+list_kappa_h)
    
    def active(u,eta):
        values = list_q_h@u + list_kappa_h
        return np.where(values<=min(values)+rho)[0]

    guarantees = []

    ###### INIT
    k = 0
    H = []
    F_check = -np.inf
    F_hat = np.inf
    condition = True
    to_check = np.arange(n)
    to_check = np.setdiff1d(to_check,[init])
    to_check = np.concatenate((np.array([init]),to_check))
    np.random.shuffle(to_check)
    selectable = [init]
    x_out = None
    
    ###### CVX
    eta_cvx,u_cvx = cp.Variable(1),cp.Variable(d)
    q_h_param,kappa_h_param = cp.Parameter(d),cp.Parameter(1)
    X_basic = [cp.sum(u_cvx)==domain_size,u_cvx>=0,eta_cvx>=0]
    if tau>0:
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) + tau/2*cp.sum_squares(u_cvx)
    else: 
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) 
    prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c@u_cvx+list_kappa_c-eta_cvx[0]<=0])
    
    ###### SOLVE
    if extra_verb:
        print('=== NEW RAM SOLVE ===')
        print(' ')
        print('-> i hat = '+str(init))
        print(' ')

    t_ref = time.time()

    ## loop
    while condition:   
        
        # -> upper-models
        R,P = np.inf,np.inf
        while len(selectable)>0:
            i_bar = np.random.choice(selectable,replace=False,size=1)[0]
            q_h_param.value,kappa_h_param.value = list_q_h[i_bar],np.array([list_kappa_h[i_bar]])
            if tau>0:
                prob.solve(solver=cp.MOSEK,warm_start=True)
            else:
                prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
            H.append(i_bar)
            if prob.value<R:
                u_bar,eta_bar = u_cvx.value,eta_cvx.value[0]
                R = F(u_bar,eta_bar)
                P = prob.value
                selectable = np.setdiff1d(active(u_bar,eta_bar),H) 
            else:
                selectable = np.setdiff1d(selectable,[i_bar])
                
        to_check = np.setdiff1d(to_check,H)
        np.random.shuffle(to_check)
        if len(to_check)>0:
            selectable = [to_check[0]]
                
        # update of upper-bound  
        if R<F_hat:
            i_out = i_bar
            F_hat = R
            print('t = '+str(time.time()-t_ref))
            print('F_hat <= '+str(R)+' for i_bar = '+str(i_bar))
            u_out,eta_out = u_bar,eta_bar
    
        guarantees.append((time.time()-t_ref,F_hat,F_check))
        
        # iteration counter increments
        k += 1
        
        # optimality or exit check
        condition = (k<MAX_ITER) and len(to_check)>0 and time.time()-t_ref<MAX_TIME
    
    print(' ')
    print('=== total time: '+str(time.time()-t_ref)+' ===')
    print('|H|/n = '+str((n-len(to_check))/n))


    if extra_verb:

        if k==MAX_ITER:
            print('/!\ EXIT: max. #iters. reached /!\ ')
        elif time.time()-t_ref>=MAX_TIME:
            print('/!\ EXIT: search time over... /!\ ')
        print(' -> final F = '+str(F(u_out,eta_out)))
    print(' ')
        
    return guarantees,u_out,eta_out

In [4]:
'''
@comment
'''
def ULO_OPO(list_q_h,list_q_c,list_kappa_h,list_kappa_c,omega=0,M=1e4,tau=0,init=0,rho=1e-3,domain_size=10,extra_verb=verb_def,\
        MAX_TIME=MAX_TIME_ABS_DEF,\
        MAX_ITER=MAX_ITER_DEF,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF):
    
    m = list_q_c.shape[0]
    n,d = list_q_h.shape
    
    ###### EVAL TOOLS
    F = lambda u,eta: tau/2*sum(u**2) + M*eta + omega*np.max(list_q_h@u+list_kappa_h) + (1-omega)*np.min(list_q_h@u+list_kappa_h)
    
    def active(u,eta):
        values = list_q_h@u + list_kappa_h
        return np.where(values<=min(values)+rho)[0]
    
    def support(u,eta):
        values = list_q_c@u + list_kappa_c - eta
        return np.where(values>=-TOL_SIGNIF_CSTR)[0]
    
    ###### STORAGE purposes
    cached_minimizers = {}
    for i in range(n):
        cached_minimizers[i] = (None,None)
    guarantees = []

    ###### INIT
    k = 0
    S = []
    H = []
    F_check = -np.inf
    F_hat = np.inf
    condition = True
    selectable = [init]
    not_converged = True
    x_out = None
    
    ###### CVX
    eta_cvx,u_cvx = cp.Variable(1),cp.Variable(d)
    q_h_param,kappa_h_param = cp.Parameter(d),cp.Parameter(1)
    X_basic = [cp.sum(u_cvx)==domain_size,u_cvx>=0,eta_cvx>=0]
    if tau>0:
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) + tau/2*cp.sum_squares(u_cvx)
    else: 
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) 
    prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c@u_cvx+list_kappa_c-eta_cvx[0]<=0])
    
    ###### SOLVE
    if extra_verb:
        print('=== NEW ULO SOLVE ===')
        print(' ')
        print('-> i hat = '+str(init))
        print(' ')

    t_ref = time.time()

    ## loop
    while condition:   
        
        # -> upper-models
        R,P = np.inf,np.inf
        while len(selectable)>0:
            i_bar = np.random.choice(selectable,replace=False,size=1)[0]
            q_h_param.value,kappa_h_param.value = list_q_h[i_bar],np.array([list_kappa_h[i_bar]])
            if tau>0:
                prob.solve(solver=cp.MOSEK,warm_start=True)
            else:
                prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
            H.append(i_bar)
            if prob.value<R:
                u_bar,eta_bar = u_cvx.value,eta_cvx.value[0]
                R = F(u_bar,eta_bar)
                P = prob.value
                selectable = np.setdiff1d(active(u_bar,eta_bar),H) 
            else:
                selectable = np.setdiff1d(selectable,[i_bar])
                
        # update of upper-bound  
        if P<F_hat:
            print('t = '+str(time.time()-t_ref))
            print('F_hat: '+str(R)+' for i_bar = '+str(i_bar))
            i_out = i_bar
            F_hat = R
            u_out,eta_out = u_bar,eta_bar
    
        guarantees.append((time.time()-t_ref,F_hat,F_check))
                            
        # -> lower-models
        new_S = list(np.setdiff1d(support(u_bar,eta_bar),S))
        S += new_S
        print('|S_'+str(k+1)+'| = '+str(len(S)))
        relaxed_prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c[S]@u_cvx+list_kappa_c[S]-eta_cvx[0]<=0])
        new_mod_val,i_hat= F_hat,i_out
        restart_set = np.setdiff1d(np.arange(n),H)
        for i in restart_set:
            q_h_param.value,kappa_h_param.value = list_q_h[i],np.array([list_kappa_h[i]])
            if cached_minimizers[i][0] is not None:
                u_cvx.value,eta_cvx.value = cached_minimizers[i][0],cached_minimizers[i][1]
            if tau>0:
                relaxed_prob.solve(solver=cp.MOSEK,warm_start=True)
            else:
                relaxed_prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
            cached_minimizers[i] = (u_cvx.value,eta_cvx.value)
            if relaxed_prob.value<=new_mod_val:
                new_mod_val = relaxed_prob.value
                i_hat = i
        print('i hat = '+str(i_hat))

        # update of lower-bound (and eventually retraced upper-bound)
        F_check = max(F_check,new_mod_val)
    
        guarantees.append((time.time()-t_ref,F_hat,F_check))
        
        not_converged = (F_hat-F_check)> max(TOL_ABS,max(1,abs(F_hat))*TOL_REL)

        selectable = [i_hat]
          
        if extra_verb:  
            print('t = '+str(time.time()-t_ref))
            print('F_hat: '+str(np.round(F_hat,4))+' & F_check: '+str(np.round(F_check,4))+'| GAP: '+str(max(0,F_hat-F_check))+' || target: '+str(max(TOL_ABS,max(1,abs(F_hat))*TOL_REL)))
        
        # iteration counter increments
        k += 1
        
        # optimality or exit check
        condition = (k<MAX_ITER) and not_converged and time.time()-t_ref<MAX_TIME
    
    print(' ')
    print('=== total time: '+str(time.time()-t_ref)+' ===')
    print('|H|/n = '+str(len(H)/n))


    if extra_verb:

        if k==MAX_ITER:
            print('/!\ EXIT: max. #iters. reached /!\ ')
        elif not_converged==False:
            print('/!\ EXIT: tolerated accuracy reached /!\ ') 
        elif time.time()-t_ref>=MAX_TIME:
            print('/!\ EXIT: search time over... /!\ ')
        print(' -> final F = '+str(F(u_out,eta_out)))
    print(' ')
        
    return guarantees,u_out,eta_out

In [5]:
'''
@comment
'''
def ES_OPO(list_q_h,list_q_c,list_kappa_h,list_kappa_c,omega=0,M=1e4,tau=0,init=0,domain_size=10,extra_verb=verb_def,\
        MAX_TIME=MAX_TIME_ABS_DEF,\
        MAX_ITER=MAX_ITER_DEF,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF):
    
    m = list_q_c.shape[0]
    n,d = list_q_h.shape
    
    ###### EVAL TOOLS
    F = lambda u,eta: tau/2*sum(u**2) + M*eta + omega*np.max(list_q_h@u+list_kappa_h) + (1-omega)*np.min(list_q_h@u+list_kappa_h)
    
    every_time_val = []
    
    ###### INIT
    k = 0
    F_hat = np.inf
    condition = True
    to_check = np.arange(n-1)
    np.random.shuffle(to_check)
    to_check = np.concatenate((np.array([init]),to_check))
    eta_out,u_out = None,None
    
    ###### CVX
    eta_cvx,u_cvx = cp.Variable(1),cp.Variable(d)
    q_h_param,kappa_h_param = cp.Parameter(d),cp.Parameter(1)
    X_basic = [cp.sum(u_cvx)==domain_size,u_cvx>=0,eta_cvx>=0]
    if tau>0:
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) + tau/2*cp.sum_squares(u_cvx)
    else: 
        obj_cvx = M*eta_cvx[0]+omega*cp.max(list_q_h@u_cvx+list_kappa_h) + (1-omega) * (q_h_param@u_cvx + kappa_h_param[0]) 
    prob = cp.Problem(cp.Minimize(obj_cvx),X_basic+[list_q_c@u_cvx+list_kappa_c-eta_cvx[0]<=0])
    
    ###### SOLVE
    if extra_verb:
        print('=== NEW ES SOLVE ===')
        print(' ')
        print('-> i hat = '+str(init))
        print(' ')

    t_ref = time.time()

    ## loop
    while condition:   
        
        # -> upper-models
        i_bar = to_check[0]
        q_h_param.value,kappa_h_param.value = list_q_h[i_bar],np.array([list_kappa_h[i_bar]])
        if tau>0:
            prob.solve(solver=cp.MOSEK,warm_start=True)
        else:
            prob.solve(solver=cp.SCIPY, scipy_options={"method": "highs"},warm_start=True)
        u_bar,eta_bar = u_cvx.value,eta_cvx.value[0]
        R = F(u_bar,eta_bar)
        
        every_time_val.append((time.time()-t_ref,R))

        to_check = to_check[1:]

        # update of upper-bound  
        if R<F_hat:
            i_out = i_bar
            F_hat = R
            print('t = '+str(time.time()-t_ref))
            print('F_hat <= '+str(R)+' for i_bar = '+str(i_bar))
            u_out,eta_out = u_bar,eta_bar
    
        # iteration counter increments
        k += 1
        
        # optimality or exit check
        condition = (k<MAX_ITER) and len(to_check)>0 and time.time()-t_ref<MAX_TIME
    
    print(' ')
    print('=== total time: '+str(time.time()-t_ref)+' ===')
    print('|H|/n = '+str((n-len(to_check))/n))

    if extra_verb:

        if k==MAX_ITER:
            print('/!\ EXIT: max. #iters. reached /!\ ')
        elif time.time()-t_ref>=MAX_TIME:
            print('/!\ EXIT: search time over... /!\ ')
        print(' -> final F = '+str(F(u_out,eta_out)))
    print(' ')
        
    return every_time_val,u_out,eta_out

### Scenarios Sampling

In [6]:
## GEOMETRY CONTROL CELL ##

dimension_global = int(100)

if name=='bign_experiment':
    m_global = int(1e3) 
    n_global = int(1e3) 

else:
    m_global = int(1e4) 
    n_global = int(5e2) 

timing = 240

ds = 10

# params
rel_impact = .15
cardinality_low,cardinality_high = int(max(1,dimension_global/10)),int(dimension_global)
ps = [1/10,1/10,3/5,1/10,1/10]

M_true = 5e4 # 1e4

omega_range = [1/2]

list_RESULTS = []
list_algorithm = []
list_rep = []
list_prob = []
list_omegas = []
list_cuts = []

num_probs = 1
num_starts = 15

maxit = np.inf

verb_ = True

tau_def = 0

cut_range = [1,5,1e-1]

In [7]:
for prob_id in range(num_probs):

    # profits
    PI_nominal = np.random.randint(0,15,dimension_global)
    PI = np.zeros((n_global,dimension_global))
    for i in range(n_global):
        PI[i] = np.maximum(0,PI_nominal + np.random.choice([-2,-1,0,1,2],\
                                                replace=True,size=dimension_global,p=ps))

    # fixed costs
    distrib_PI = np.sum(PI,1)
    PHI = np.random.uniform(0,max(distrib_PI)-min(distrib_PI),n_global)*rel_impact

    importance_nominal = np.argsort(PI_nominal)

    ## investments constraints
    weights = np.array([float(elem)**5 for elem in np.arange(dimension_global)])
    weights /= np.sum(weights)
    MU,TAU = np.zeros((m_global,dimension_global)),np.zeros(m_global)
    for j in range(m_global):
        cardinality = np.random.randint(cardinality_low,cardinality_high)
        idx = np.random.choice(importance_nominal,p=weights,size=cardinality,replace=False)
        buf = np.random.randint(1,11,len(idx))
        MU[j,idx] = buf
        TAU[j] = sum(MU[j,idx])*cardinality/dimension_global
    
    for cut in cut_range:
        
        print('cut = '+str(cut))
        print(' ')
        
        TAU_cut = TAU*cut # param set
        
        for omega in omega_range:
            
            print('omega = '+str(omega))
        
            counter = 0

            for id_start in list(np.random.choice(np.arange(n_global),size=num_starts,replace=False)):
                counter += 1
                print(' ')
                print(' start| '+str(counter))
                print(' ')
                
                guar_ram,eta_out_ram,u_out_ram = RAM_OPO(list_q_h=-PI,list_q_c=MU,list_kappa_h=PHI,list_kappa_c=-TAU_cut,omega=omega,M=M_true,tau=tau_def,init=id_start,rho=1e-3,domain_size=ds,extra_verb=verb_,\
            MAX_TIME=timing,\
            MAX_ITER=np.inf,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF)

                list_RESULTS.append(guar_ram)
                list_algorithm.append('RAM')
                list_prob.append(prob_id)
                list_rep.append(counter)
                list_omegas.append(omega)
                list_cuts.append(cut)

                guar_ulo,eta_out_ulo,u_out_ulo = ULO_OPO(list_q_h=-PI,list_q_c=MU,list_kappa_h=PHI,list_kappa_c=-TAU_cut,omega=omega,M=M_true,tau=tau_def,init=id_start,rho=1e-3,domain_size=ds,extra_verb=verb_,\
            MAX_TIME=timing,\
            MAX_ITER=maxit,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF)

                list_RESULTS.append(guar_ulo)
                list_algorithm.append('ULO')
                list_prob.append(prob_id)
                list_rep.append(counter)
                list_omegas.append(omega)
                list_cuts.append(cut)

            time_val_tuples_es,eta_out_es,u_out_es = ES_OPO(list_q_h=-PI,list_q_c=MU,list_kappa_h=PHI,list_kappa_c=-TAU_cut,omega=omega,M=M_true,tau=tau_def,init=id_start,domain_size=ds,extra_verb=verb_,\
        MAX_TIME=np.inf,\
        MAX_ITER=np.inf,TOL_REL=TOL_REL_DEF,TOL_ABS=TOL_ABS_DEF,TOL_SIGNIF_CSTR=TOL_SIGNIF_CSTR_DEF)

            list_RESULTS.append(time_val_tuples_es)
            list_algorithm.append('ES')
            list_prob.append(prob_id)
            list_rep.append(counter)
            list_omegas.append(omega)
            list_cuts.append(cut)
            
            BOUNDs,btypes,Ts,ALG,CUT,OMEGA,PROBID,REPID = [],[],[],[],[],[],[],[]
            for res,alg,pbid,repid,cu,om in zip(list_RESULTS,list_algorithm,list_prob,list_rep,list_cuts,list_omegas):
                for line in res:
                    if alg=='ES':
                        BOUNDs += [line[1],np.inf]
                    else:
                        BOUNDs += [line[1],line[2]]
                    btypes += ['UB','LB']
                    Ts += [line[0],line[0]]
                    ALG += [alg,alg]
                    CUT += [cu,cu]
                    OMEGA += [om,om]
                    PROBID += [pbid,pbid]
                    REPID += [repid,repid]
                    
            experiment_df = pd.DataFrame(data={'cut':CUT,'omega':OMEGA,'algo':ALG,'times':Ts,'bounds':BOUNDs,'bound_type':btypes,\
                                           'probID':PROBID,'repID':REPID})
            
            experiment_df.to_csv(name+'_nopta_bis_3.csv')
        
    print(' ')

cut = 1
 
omega = 0.5
 
 start| 1
 
=== NEW RAM SOLVE ===
 
-> i hat = 744
 
t = 0.3660013675689697
F_hat <= -87.33343287888772 for i_bar = 608
t = 1.0060036182403564
F_hat <= -87.36517139813147 for i_bar = 67
t = 1.888007640838623
F_hat <= -87.72887686927012 for i_bar = 396
 
=== total time: 147.34057521820068 ===
|H|/n = 1.0
 -> final F = -87.72887686927012
 
=== NEW ULO SOLVE ===
 
-> i hat = 744
 
t = 0.35500073432922363
F_hat: -87.33343287888772 for i_bar = 608
|S_1| = 28
i hat = 396
t = 51.31722354888916
F_hat: -87.3334 & F_check: -89.8441| GAP: 2.5106349101843364 || target: 0
t = 51.46622371673584
F_hat: -87.72887686927012 for i_bar = 396
|S_2| = 36
i hat = 7
t = 104.49043369293213
F_hat: -87.7289 & F_check: -88.1883| GAP: 0.4594438407920194 || target: 0
|S_3| = 42
i hat = 396
t = 157.48262286186218
F_hat: -87.7289 & F_check: -87.7289| GAP: 0 || target: 0
 
=== total time: 157.48262286186218 ===
|H|/n = 0.004
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -87.7288768692

i hat = 67
t = 51.645612716674805
F_hat: -87.3257 & F_check: -90.1541| GAP: 2.8284155847298393 || target: 0
t = 51.80961632728577
F_hat: -87.36517139813147 for i_bar = 67
|S_2| = 42
i hat = 608
t = 103.5092282295227
F_hat: -87.3652 & F_check: -88.2482| GAP: 0.882987766026389 || target: 0
|S_3| = 45
i hat = 396
t = 157.0868637561798
F_hat: -87.3652 & F_check: -87.9108| GAP: 0.5455990178320604 || target: 0
t = 157.2608664035797
F_hat: -87.72887686927012 for i_bar = 396
|S_4| = 47
i hat = 396
t = 212.62252354621887
F_hat: -87.7289 & F_check: -87.7289| GAP: 0 || target: 0
 
=== total time: 212.62252354621887 ===
|H|/n = 0.005
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -87.72887686927012
 
 
 start| 10
 
=== NEW RAM SOLVE ===
 
-> i hat = 60
 
t = 0.4160044193267822
F_hat <= -87.32566942844718 for i_bar = 7
t = 0.7510080337524414
F_hat <= -87.33343287888772 for i_bar = 608
t = 2.540029764175415
F_hat <= -87.36223766045327 for i_bar = 144
t = 3.2380380630493164
F_hat <= -87.7288

i hat = 835
t = 87.82743406295776
F_hat: -130.8444 & F_check: -131.0305| GAP: 0.18615686165813372 || target: 0
|S_3| = 11
i hat = 94
t = 133.68720078468323
F_hat: -130.8444 & F_check: -130.8444| GAP: 0 || target: 0
 
=== total time: 133.68720078468323 ===
|H|/n = 0.004
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -130.84439085652022
 
 
 start| 2
 
=== NEW RAM SOLVE ===
 
-> i hat = 248
 
t = 0.4050111770629883
F_hat <= -129.6811175469133 for i_bar = 835
t = 0.6960194110870361
F_hat <= -130.11230453216592 for i_bar = 970
t = 1.0010287761688232
F_hat <= -130.3129860677172 for i_bar = 517
t = 1.4390404224395752
F_hat <= -130.84439085652022 for i_bar = 94
 
=== total time: 149.01510787010193 ===
|H|/n = 1.0
 -> final F = -130.84439085652022
 
=== NEW ULO SOLVE ===
 
-> i hat = 248
 
t = 0.38201022148132324
F_hat: -129.6811175469133 for i_bar = 835
|S_1| = 7
i hat = 94
t = 43.94010877609253
F_hat: -129.6811 & F_check: -132.269| GAP: 2.5878974511974207 || target: 0
t = 44.0991132

i hat = 517
t = 90.86368775367737
F_hat: -130.8444 & F_check: -132.1382| GAP: 1.2937849992597705 || target: 0
|S_3| = 12
i hat = 94
t = 135.45702576637268
F_hat: -130.8444 & F_check: -130.8444| GAP: 0 || target: 0
 
=== total time: 135.45702576637268 ===
|H|/n = 0.004
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -130.84439085652022
 
 
 start| 11
 
=== NEW RAM SOLVE ===
 
-> i hat = 529
 
t = 0.3780043125152588
F_hat <= -130.84439085652022 for i_bar = 94
 
=== total time: 155.6951539516449 ===
|H|/n = 1.0
 -> final F = -130.84439085652022
 
=== NEW ULO SOLVE ===
 
-> i hat = 529
 
t = 0.37200260162353516
F_hat: -130.84439085652022 for i_bar = 94
|S_1| = 6
i hat = 517
t = 44.156286001205444
F_hat: -130.8444 & F_check: -132.9381| GAP: 2.093668599596782 || target: 0
|S_2| = 8
i hat = 835
t = 88.30544638633728
F_hat: -130.8444 & F_check: -131.0305| GAP: 0.18615686165813372 || target: 0
|S_3| = 11
i hat = 94
t = 135.60475277900696
F_hat: -130.8444 & F_check: -130.8444| GAP: 0 || 

t = 1.3910253047943115
F_hat <= -29.182954046081328 for i_bar = 8
 
=== total time: 161.55980610847473 ===
|H|/n = 1.0
 -> final F = -29.182954046081328
 
=== NEW ULO SOLVE ===
 
-> i hat = 941
 
t = 0.3980066776275635
F_hat: -28.46673606724554 for i_bar = 920
|S_1| = 60
i hat = 848
t = 64.34411835670471
F_hat: -28.4667 & F_check: -31.3206| GAP: 2.85388485588917 || target: 0
|S_2| = 84
i hat = 556
t = 134.32033133506775
F_hat: -28.4667 & F_check: -30.1944| GAP: 1.7276780525916955 || target: 0
t = 134.47533297538757
F_hat: -28.86070542717566 for i_bar = 556
|S_3| = 99
i hat = 8
t = 206.92640328407288
F_hat: -28.8607 & F_check: -29.5469| GAP: 0.6861682437913821 || target: 0
t = 207.09540510177612
F_hat: -29.182954046081328 for i_bar = 8
|S_4| = 112
i hat = 8
t = 281.91551089286804
F_hat: -29.183 & F_check: -29.183| GAP: 0 || target: 0
 
=== total time: 281.91551089286804 ===
|H|/n = 0.005
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -29.182954046081328
 
 
 start| 4
 
=== NEW 

t = 1.798004388809204
F_hat <= -29.182954046081328 for i_bar = 8
 
=== total time: 158.98401021957397 ===
|H|/n = 1.0
 -> final F = -29.182954046081328
 
=== NEW ULO SOLVE ===
 
-> i hat = 676
 
t = 0.38699960708618164
F_hat: -28.75048996872093 for i_bar = 452
|S_1| = 53
i hat = 70
t = 61.63115692138672
F_hat: -28.7505 & F_check: -30.8454| GAP: 2.094953103830939 || target: 0
|S_2| = 80
i hat = 8
t = 130.57733368873596
F_hat: -28.7505 & F_check: -29.7074| GAP: 0.956872022157981 || target: 0
t = 130.73133492469788
F_hat: -29.182954046081328 for i_bar = 8
|S_3| = 96
i hat = 556
t = 203.06851983070374
F_hat: -29.183 & F_check: -29.2593| GAP: 0.0763890520395023 || target: 0
|S_4| = 107
i hat = 8
t = 279.66970896720886
F_hat: -29.183 & F_check: -29.183| GAP: 0 || target: 0
 
=== total time: 279.66970896720886 ===
|H|/n = 0.005
/!\ EXIT: tolerated accuracy reached /!\ 
 -> final F = -29.182954046081328
 
 
 start| 12
 
=== NEW RAM SOLVE ===
 
-> i hat = 222
 
t = 0.41100120544433594
F_hat <= 